In [1]:
import sys
sys.path.append('/home/nick/python/asop_global/ASoP-Coherence')
import asop_coherence as asop
from asop_coherence_global_temporal import load_cmip6,mask_wet_season,mask_min_precip
import iris
from pathlib import Path
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import iris.coord_categorisation
from iris.experimental.equalise_cubes import equalise_attributes
from iris.util import unify_time_units
import dask
from dask.distributed import Client,progress

In [2]:
def haversine(origin, destination):
    import math

    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

In [3]:
def get_asop_dict(key):
    cmip6_path=Path('/media/nick/lacie_tb3/data_from_gill/CMIP6')
    obs_path=Path('/media/nick/data')
    if key == 'GPM_IMERG':
        asop_dict={
            'desc': '3B-HHR.MS.MRG.3IMERG.V06B.3hr_means_3x3',
            'dir': obs_path/'GPM_IMERG',
            'file_pattern': '3B-HHR.MS.MRG.3IMERG.*.3hr_means_3x3.V06B.nc',
            'name': 'IMERG-3B-V06',
            'start_year': 2001,
            'stop_year': 2001,
            'dt': 10800,
            'legend_name': 'IMERG',
            'region': [-60,60,0,360],
            'color': 'black',
            'symbol': '>',
            'region_size': 3,
            'lag_length': 2,
            'dx': 330,
            'dy': 330
        }
    else:
        raise Exception('No dictionary for '+key)
    return(asop_dict)

In [12]:
def compute_equalgrid_corr_global(precip,distance_bins):
    longitude = precip.coord('longitude')
    nlon=len(longitude.points)
    latitude = precip.coord('latitude')
    nlat=len(latitude.points)
    time = precip.coord('time')
    ntime=len(time.points)
    nbins = len(distance_bins)-1
    dist_centre = np.zeros(nbins)
    min_dist = np.zeros(nbins)
    max_dist = np.zeros(nbins)
    bounds = np.zeros((nbins,2))
    for b,left in enumerate(distance_bins[0:-1]):
        min_dist[b] = left
        max_dist[b] = distance_bins[b+1]
        dist_centre[b] = (max_dist[b]+min_dist[b])/2.0
        bounds[b,:] = np.asarray((min_dist[b],max_dist[b]))
    distance = iris.coords.DimCoord(dist_centre,var_name='distance',bounds=bounds)
    distance_corrs = iris.cube.Cube(np.zeros((nbins,nlat,nlon)),var_name='distance_correlations',dim_coords_and_dims=[(distance,0),(latitude,1),(longitude,2)])

    for y,latpt in enumerate(latitude.points):
        dask_distcorr=[]
        for x,lonpt in enumerate(longitude.points):
            for b in range(nbins):
                precip_mask = extract_mask_region(precip,latpt,lonpt,min_dist[b],max_dist[b])
                this_distcorr = dask.delayed(compute_gridcorr_grid)(precip[:,y,x],precip_mask)
                dask_distcorr.append(this_distcorr)
        result = dask.compute(*dask_distcorr)
        result = np.ma.asarray(result)
        print(np.shape(result))
    return(distance_corrs)


In [32]:
def extract_mask_region(precip,centre_lat,centre_lon,dist_min,dist_max):
    longitude = precip.coord('longitude')
    nlon=len(longitude.points)
    latitude = precip.coord('latitude')
    nlat=len(latitude.points)
    time = precip.coord('time')
    ntime=len(time.points)
    pt_dist = np.ones((nlat,nlon))
    dN = 0 ; dS = 0 ; dE = 0 ; dW = 0
    for yy,target_lat in enumerate(latitude.points):
        for xx,target_lon in enumerate(longitude.points):
            pt_dist[yy,xx] = haversine((centre_lat,centre_lon),(target_lat,target_lon))
            if pt_dist[yy,xx] <= dist_max:
                dN = np.amax([dN,target_lat-centre_lat])
                dS = np.amin([dS,target_lat-centre_lat])
                dE = np.amax([dE,target_lon-centre_lon])
                dW = np.amin([dW,target_lon-centre_lon])
    print(centre_lon,centre_lat,dN,dS,dE,dW,centre_lon+dW,centre_lon+dE)
#    if lonmin <= c
    subset = precip.intersection(longitude = (centre_lon+dW,centre_lon+dE),latitude=(centre_lat-dS,centre_lat+dN))
    print(subset)
    dist_mask = np.ones_like(subset.data)
    for yy,target_lat in enumerate(subset.coord('latitude').points):
        for xx,target_lon in enumerate(subset.coord('longitude').points):
            pt_dist = haversine((centre_lat,centre_lon),(target_lat,target_lon))
            if pt_dist >= dist_min and pt_dist <= dist_max:
                dist_mask.data[:,yy,xx] = 0
    subset_mask = subset.copy(data=np.ma.array(subset.data,mask=dist_mask))
    return(subset_mask)

In [6]:
def compute_gridcorr_grid(precip,grid):
    import iris.analysis.stats as istats
    corr_map = istats.pearsonr(precip,grid,corr_coords='time')
    weights = iris.analysis.cartography.area_weights(corr_map)
    output=corr_map.collapsed(['longitude','latitude'],iris.analysis.MEAN,weights=weights)
    return(output.data)

In [7]:
client = Client()
dataset='GPM_IMERG'
asop_dict = get_asop_dict(dataset)

In [8]:
precip = load_cmip6(asop_dict)
masked_precip = mask_wet_season(precip)
masked_precip = mask_min_precip(masked_precip)

In [33]:
test = compute_equalgrid_corr_global(masked_precip,[0,450,900,1350])

1.5 -61.5 3.0 0.0 357.0 0.0 1.5 358.5
precipitation_flux / (mm/hr)        (time: 2920; latitude: 2; longitude: 120)
     Dimension coordinates:
          time                           x               -             -
          latitude                       -               x             -
          longitude                      -               -             x
     Auxiliary coordinates:
          hour                           x               -             -
          month_number                   x               -             -
          year                           x               -             -
     Attributes:
          NCO: netCDF Operators version 4.8.1 (Homepage = http://nco.sf.net, Code = h...
     Cell methods:
          mean: time
1.5 -61.5 6.0 0.0 357.0 0.0 1.5 358.5
precipitation_flux / (mm/hr)        (time: 2920; latitude: 3; longitude: 120)
     Dimension coordinates:
          time                           x               -             -
          latitude         

KeyboardInterrupt: 

In [10]:
print(test)

NameError: name 'test' is not defined

In [11]:
corr_map,lag_vs_distance,autocorr,npts_map,npts = asop.compute_equalgrid_corr(masked_precip,asop_dict)

---> Computing correlations for 3x3 sub-regions
----> Info: Size of domain in native gridpoints: 120 longitude x 42 latitude.
----> Info: There are 560 3x3 sub-regions in your input data.
